In [2]:
import json
import random 

data_file = "res_summary/4_1_llm_test_results_CATI-AI_CMC-Legal-LLM-32B-sft-v2.jsonl"

with open(data_file, "r") as f:
    lines = [json.loads(line) for line in f.readlines()]

random_sample = random.choice(lines)
print(random_sample['content'])
# print(random_sample["ground_truth"])
# print("*" * 60) 
# print(random_sample["prediction"])

NGHỊ ĐỊNH
SỬA ĐỔI, BỔ SUNG MỘT SỐ ĐIỀU CỦA NGHỊ ĐỊNH SỐ 41/2014/NĐ-CP NGÀY 13 THÁNG 5 NĂM 2014 CỦA CHÍNH PHỦ QUY ĐỊNH VỀ TỔ CHỨC VÀ HOẠT ĐỘNG THANH TRA CÔNG AN NHÂN DÂN
Căn cứ Luật Tổ chức Chính phủ ngày 19 tháng 6 năm 2015; Luật sửa đổi, bổ sung một số điều của Luật Tổ chức Chính phủ và Luật Tổ chức chính quyền địa phương ngày 22 tháng 11 năm 2019;
Căn cứ Luật Thanh tra ngày 15 tháng 11 năm 2010;
Căn cứ Luật Công an nhân dân ngày 20 tháng 11 năm 2018;
Theo đề nghị của Bộ trưởng Bộ Công an;
Chính phủ ban hành Nghị định sửa đổi, bổ sung một số điều của Nghị định số 41/2014/NĐ-CP ngày 13 tháng 5 năm 2014 của Chính phủ quy định về tổ chức và hoạt động thanh tra Công an nhân dân.
Điều 1. Sửa đổi, bổ sung một số điều của Nghị định số 41/2014/NĐ-CP
1. Khoản 1, khoản 3 Điều 3 được sửa đổi, bổ sung như sau:
“1. Hệ thống cơ quan thanh tra nhà nước trong Công an nhân dân, gồm:
a) Thanh tra Bộ Công an (sau đây gọi là Thanh tra Bộ);
b) Thanh tra Công an tỉnh, thành phố trực thuộc Trung ương (sau đ

In [ ]:
from test_fc.prompt import prompt
import logging 
import os 
import json 
from openai import OpenAI
def function_calling(query: str) -> dict:
    """
    Perform function calling to extract essential terms
    """
    FC_PAYLOAD_TOOLCALL = "function_calling_toolcall.json"
    with open(FC_PAYLOAD_TOOLCALL, "r") as f:
        function_def = json.load(f)

    # Wrap the function definition in the correct format
    tools = [
        {
            "type": "function",
            "function": function_def
        }
    ]

    # Init OpenAI client
    api_key = os.getenv("API_KEY")

    client = OpenAI(
        api_key=api_key, 
        base_url=f"{os.getenv('HOST_NAME')}/v1",
    )

    response = client.chat.completions.create(
        model="CATI-AI/CMC-Legal-LLM-32B-sft-v2",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": query}
        ],
        tools=tools,
        tool_choice={
            "type": "function",
            "function": {"name": "extract_intent_and_query"}
        }
    )

    # Log the raw response for debugging
    arguments_str = response.choices[0].message.tool_calls[0].function.arguments
    # logging.info(f"Raw arguments_str from LLM: '{arguments_str}'")
    
    # Handle empty or invalid JSON
    if not arguments_str or arguments_str.strip() == "":
        logging.error(f"Empty arguments returned for query: {query}")
        return {
            "query": query,
            "loai_van_ban": "",
            "chu_de_phap_dien": "",
            "tinh_trang_hieu_luc": []
        }
    
    # Flexible parsing - handle multiple formats
    original_str = arguments_str
    
    # Strip common junk prefixes/suffixes
    arguments_str = arguments_str.strip()
    
    # Format 0: Handle weird prefixes like "<>", "<name:...", "<function>", "<function-call>", etc.
    # Remove leading tags that are not valid JSON
    if arguments_str.startswith("<"):
        # Find the first occurrence of "{" which likely starts the actual JSON
        json_start = arguments_str.find("{")
        if json_start != -1:
            arguments_str = arguments_str[json_start:].strip()
    
    # Format 1: Handle "{Name}: ..., {Arguments}: {...}" or similar patterns
    if ("{Name}:" in arguments_str or "{name}:" in arguments_str) and ("{Arguments}:" in arguments_str or "{arguments}:" in arguments_str):
        # Extract everything after "{Arguments}:" or "{arguments}:"
        for pattern in ["{Arguments}:", "{arguments}:"]:
            if pattern in arguments_str:
                args_start = arguments_str.find(pattern) + len(pattern)
                arguments_str = arguments_str[args_start:].strip()
                break
    
    # Format 2: Handle "<name>: ..., <arguments>: {...}"
    elif ("<name>:" in arguments_str or "<Name>:" in arguments_str) and "<arguments>:" in arguments_str:
        args_start = arguments_str.find("<arguments>:") + len("<arguments>:")
        arguments_str = arguments_str[args_start:].strip()
    
    # Format 3: Handle XML-like tags like <arguments>...</arguments>
    elif "<arguments>" in arguments_str and "</arguments>" in arguments_str:
        start = arguments_str.find("<arguments>") + len("<arguments>")
        end = arguments_str.find("</arguments>")
        arguments_str = arguments_str[start:end].strip()
    
    # Format 4: Handle <function>...</function>
    elif "<function>" in arguments_str and "</function>" in arguments_str:
        start = arguments_str.find("<function>") + len("<function>")
        end = arguments_str.find("</function>")
        arguments_str = arguments_str[start:end].strip()
    
    # Format 5: Handle <extract_intent_and_query>...</extract_intent_and_query>
    elif "<extract_intent_and_query>" in arguments_str:
        if "</extract_intent_and_query>" in arguments_str:
            start = arguments_str.find("<extract_intent_and_query>") + len("<extract_intent_and_query>")
            end = arguments_str.find("</extract_intent_and_query>")
            arguments_str = arguments_str[start:end].strip()
        else:
            # Remove the opening tag and keep the rest
            arguments_str = arguments_str.replace("<extract_intent_and_query>", "").strip()
    
    # Format 6: Handle <name>xxx</name>\n<arguments>...</arguments>
    elif "<name>" in arguments_str and "</name>" in arguments_str:
        if "<arguments>" in arguments_str and "</arguments>" in arguments_str:
            start = arguments_str.find("<arguments>") + len("<arguments>")
            end = arguments_str.find("</arguments>")
            arguments_str = arguments_str[start:end].strip()
    
    # Format 7: If it starts with "{"name": and contains "arguments", extract just the arguments
    if arguments_str.startswith('{"name":') or arguments_str.startswith("{'name':"):
        try:
            temp_parse = json.loads(arguments_str)
            if isinstance(temp_parse, dict) and "arguments" in temp_parse:
                arguments_str = json.dumps(temp_parse["arguments"])
        except:
            pass
    
    # Format 8: If it's just plain text like "function-call", return default
    if arguments_str.strip() in ["function-call", "function_call", "<tool>", "</tool>", "<>", ""]:
        logging.error(f"Invalid response format for query '{query}': '{original_str[:200]}'")
        return {
            "query": query,
            "loai_van_ban": "",
            "chu_de_phap_dien": "",
            "tinh_trang_hieu_luc": []
        }
    
    try:
        parsed = json.loads(arguments_str)
        
        # If the JSON has an "arguments" key, use that as the actual arguments
        if isinstance(parsed, dict) and "arguments" in parsed:
            arguments = parsed["arguments"]
            # logging.info(f"Extracted nested arguments: {arguments}")
        else:
            arguments = parsed
            
    except json.JSONDecodeError as e:
        logging.error(f"JSON decode error for query '{query}': {e}. Tried: '{arguments_str[:200]}'")
        # Return default payload
        return {
            "query": query,
            "loai_van_ban": "",
            "chu_de_phap_dien": "",
            "tinh_trang_hieu_luc": []
        }
    return arguments

In [12]:
import os 
import json 
gt_folder = "data_test_fc/gpt"

for log in os.listdir(gt_folder):
    with open(os.path.join(gt_folder, log), "r") as f:
        data = json.load(f)
    
    user_query = data["request_params"]["messages"][0]["content"]
    print(user_query)
    
    gpt_payload = data["query_params"]
    del gpt_payload["_metadata"]
    print(gpt_payload)
    
    # Get payload from function calling
    fc_payload = function_calling(user_query)
    print(fc_payload)
    
    # Compare EACH key of gpt_payload and fc_payload
    for key in gpt_payload:
        if key not in fc_payload:
            print(f"Key '{key}' missing in FC payload")
            continue
    break

Quyền sở hữu trí tuệ bao gồm những quyền gì?
{'use_memory': True, 'intent': 'legal_query', 'reference_text': '', 'ten_van_ban': 'luật sở hữu trí tuệ', 'so_hieu': '36/2009/QH12', 'tinh_trang_hieu_luc': ['Còn hiệu lực'], 'loai_van_ban': ['Luật,Bộ luật'], 'co_quan_ban_hanh': 'Quốc hội', 'ngay_ban_hanh_start': '', 'ngay_ban_hanh_end': '', 'ngay_co_hieu_luc_start': '', 'ngay_co_hieu_luc_end': '', 'loai_hinh_van_ban': 'văn bản quy phạm', 'query': 'quyền tác giả quyền liên quan quyền tác giả luật sở hữu trí tuệ 36/2009/QH12', 'query_external_search': '', 'need_subquery': False, 'retrieve_level': 'term', 'use_rag': True, 'term_position': []}
{'use_memory': False, 'intent': 'legal_query', 'use_rag': True, 'reference_text': '', 'ten_van_ban': '', 'tinh_trang_hieu_luc': [], 'loai_van_ban': [], 'co_quan_ban_hanh': '', 'ngay_ban_hanh_start': '', 'ngay_ban_hanh_end': '', 'ngay_co_hieu_luc_start': '', 'ngay_co_hieu_luc_end': '', 'loai_hinh_van_ban': '', 'query': 'quyền sở hữu trí tuệ bao gồm quyền gì

In [3]:
def _finalize_chunk(chunk):
    """Hoàn thiện chunk trước khi thêm vào danh sách."""
    return {
        'content': chunk['content'],
        'articles': chunk['articles'],
        'doc_ids': list(chunk['doc_ids']),
        'start_position': chunk['start_position'],
        'end_position': chunk['end_position'],
        'length': len(chunk['content']),
        'article_count': len(chunk['articles'])
    }

def _create_empty_chunk():
    """Tạo chunk rỗng."""
    return {
        'content': '',
        'articles': [],
        'doc_ids': set(),
        'start_position': '',
        'end_position': ''
    }

def chunk_legal_documents(articles, max_length=1000, separator="\n\n"):
    """
    Gộp các điều khoản pháp luật thành chunks với độ dài tối đa.
   
    Args:
        articles (list[dict]): Danh sách các điều khoản
        max_length (int): Độ dài tối đa của mỗi chunk
        separator (str): Ký tự phân cách giữa các điều khoản
   
    Returns:
        list[dict]: Danh sách các chunks
    """
    if not articles:
        return []
   
    chunks = []
    current_chunk = {
        'content': '',
        'articles': [],
        'doc_ids': set(),
        'start_position': '',
        'end_position': ''
    }
   
    separator_len = len(separator)
   
    for article in articles:
        # Sử dụng content nếu có, không thì dùng title
        article_text = article.get('content', '').strip() or article.get('title', '').strip()
       
        if not article_text:
            continue
           
        # Nếu điều khoản đơn lẻ đã vượt quá max_length
        if len(article_text) > max_length:
            # Lưu chunk hiện tại nếu có nội dung
            if current_chunk['content']:
                chunks.append(_finalize_chunk(current_chunk))
                current_chunk = _create_empty_chunk()
           
            # Tạo chunk riêng cho điều khoản dài
            chunks.append({
                'content': article_text,
                'articles': [article],
                'doc_ids': [article.get('doc_ids')],
                'start_position': article.get('position', ''),
                'end_position': article.get('position', ''),
                'length': len(article_text)
            })
            continue
       
        # Tính độ dài nếu thêm điều khoản này
        additional_length = len(article_text)
        if current_chunk['content']:
            additional_length += separator_len
           
        new_total_length = len(current_chunk['content']) + additional_length
       
        # Nếu vượt quá max_length, lưu chunk hiện tại và bắt đầu chunk mới
        if new_total_length > max_length and current_chunk['content']:
            chunks.append(_finalize_chunk(current_chunk))
            current_chunk = _create_empty_chunk()
       
        # Thêm điều khoản vào chunk hiện tại
        if current_chunk['content']:
            current_chunk['content'] += separator + article_text
        else:
            current_chunk['content'] = article_text
            current_chunk['start_position'] = article.get('position', '')
           
        current_chunk['articles'].append(article)
        current_chunk['doc_ids'].add(article.get('doc_ids'))
        current_chunk['end_position'] = article.get('position', '')
   
    # Lưu chunk cuối cùng nếu có nội dung
    if current_chunk['content']:
        chunks.append(_finalize_chunk(current_chunk))
   
    return chunks

In [4]:
# Test method chunk_legal_documents
articles = [
    {
      "index": 2,
      "id": "dieu_1",
      "id_document": 147168,
      "content": "QUY ĐỊNH CHUNG\nPhạm vi điều chỉnh Nghị định này quy định chi tiết một số điều của Nghị quyết số 132/2020/QH14 ngày 17 tháng 11 năm 2020 của Quốc hội thí điểm một số chính sách để tháo gỡ vướng mắc, tồn đọng trong quản lý, sử dụng đất quốc phòng, an ninh kết hợp với hoạt động lao động sản xuất, xây dựng kinh tế (sau đây gọi là Nghị quyết số 132/2020/QH14), bao gồm các nội dung sau:\nChế độ sử dụng đất quốc phòng, an ninh kết hợp với hoạt động lao động sản xuất, xây dựng kinh tế. Xử lý đất quốc phòng, an ninh đối với dự án, hợp đồng thuê đất, hợp đồng liên doanh, liên kết đã thực hiện và đối với doanh nghiệp cổ phần hóa, thoái vốn.",
      "raw_content": "Chương I\nQUY ĐỊNH CHUNG\n ... \nĐiều 1. Phạm vi điều chỉnh\nNghị định này quy định chi tiết một số điều của Nghị quyết số 132/2020/QH14 ngày 17 tháng 11 năm 2020 của Quốc hội thí điểm một số chính sách để tháo gỡ vướng mắc, tồn đọng trong quản lý, sử dụng đất quốc phòng, an ninh kết hợp với hoạt động lao động sản xuất, xây dựng kinh tế (sau đây gọi là Nghị quyết số 132/2020/QH14), bao gồm các nội dung sau:\n1. Chế độ sử dụng đất quốc phòng, an ninh kết hợp với hoạt động lao động sản xuất, xây dựng kinh tế.\n2. Xử lý đất quốc phòng, an ninh đối với dự án, hợp đồng thuê đất, hợp đồng liên doanh, liên kết đã thực hiện và đối với doanh nghiệp cổ phần hóa, thoái vốn.",
      "position": "Điều 1 / Chương I",
      "title": "Điều 1. Phạm vi điều chỉnh\nNghị định này quy định chi tiết một số điều của Nghị quyết số 132/2020/QH14 ngày 17 tháng 11 năm 2020 của Quốc hội thí điểm một số chính sách để tháo gỡ vướng mắc, tồn đọng trong quản lý, sử dụng đất quốc phòng, an ninh kết hợp với hoạt động lao động sản xuất, xây dựng kinh tế (sau đây gọi là Nghị quyết số 132/2020/QH14), bao gồm các nội dung sau:",
      "num_words": 0,
      "insert_at": "2025-11-17T16:23:01.970716",
      "dia_danh": "Trung ương",
      "ngay_ban_hanh": "2021-03-25T00:00:00Z",
      "ngay_co_hieu_luc": "2021-03-25T00:00:00Z",
      "ngay_dang_cong_bao": "2021-04-06T00:00:00Z",
      "ngay_het_hieu_luc": "2025-08-15T00:00:00Z",
      "ngay_het_hieu_luc_mot_phan": None,
      "so_hieu": "26/2021/NĐ-CP",
      "trich_yeu": "Hướng dẫn Nghị quyết 132/2020/QH14 thí điểm chính sách đất quốc phòng",
      "tinh_trang_hieu_luc": "Hết hiệu lực toàn bộ",
      "co_quan_ban_hanh": "Chính phủ",
      "nguoi_ky": "Nguyễn Xuân Phúc",
      "loai_van_ban": "Nghị định",
      "don_vi": [
        "Bộ Quốc phòng",
        "Trung ương"
      ],
      "linh_vuc": "Bất động sản",
      "url": None,
      "type_of_van_ban": 1,
      "score": 1
    }
  ]

chunks = chunk_legal_documents(articles, max_length=500)
for chunk in chunks:
    print(chunk)
    print("-----")

{'content': 'QUY ĐỊNH CHUNG\nPhạm vi điều chỉnh Nghị định này quy định chi tiết một số điều của Nghị quyết số 132/2020/QH14 ngày 17 tháng 11 năm 2020 của Quốc hội thí điểm một số chính sách để tháo gỡ vướng mắc, tồn đọng trong quản lý, sử dụng đất quốc phòng, an ninh kết hợp với hoạt động lao động sản xuất, xây dựng kinh tế (sau đây gọi là Nghị quyết số 132/2020/QH14), bao gồm các nội dung sau:\nChế độ sử dụng đất quốc phòng, an ninh kết hợp với hoạt động lao động sản xuất, xây dựng kinh tế. Xử lý đất quốc phòng, an ninh đối với dự án, hợp đồng thuê đất, hợp đồng liên doanh, liên kết đã thực hiện và đối với doanh nghiệp cổ phần hóa, thoái vốn.', 'articles': [{'index': 2, 'id': 'dieu_1', 'id_document': 147168, 'content': 'QUY ĐỊNH CHUNG\nPhạm vi điều chỉnh Nghị định này quy định chi tiết một số điều của Nghị quyết số 132/2020/QH14 ngày 17 tháng 11 năm 2020 của Quốc hội thí điểm một số chính sách để tháo gỡ vướng mắc, tồn đọng trong quản lý, sử dụng đất quốc phòng, an ninh kết hợp với ho